In [2]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el modelo
modelo_path = 'Modelo_ALAVA.pkl'  # Cambia esta ruta a la correcta
modelo = joblib.load(modelo_path)


datos_entrada = {
    "TMEDIA": 5.31125,
    "TMIN": 1.3264919354838711,
    "TMAX": 9.296043906810036,
    "VELMEDIA": 2.4746397849462367,
    "SOL": 3.3099999999999996,
    "PRESMAX": 965.4137096774193,
    "PRESMIN": 960.1672580645161,
    "Potencia contratada (kW)": 2,  
    "Mes": 1,  
    "Media de residentes": 4,  
    "Tipo de vivienda_Adosado": False,
    "Tipo de vivienda_Casa Unifamiliar": False,
    "Tipo de vivienda_Duplex": False,
    "Tipo de vivienda_Piso": True  
}

feature_names = [
    "TMEDIA", "TMIN", "TMAX", "VELMEDIA", "SOL", "PRESMAX", "PRESMIN",
    "Potencia contratada (kW)", "Mes", "Media de residentes",
    "Tipo de vivienda_Adosado", "Tipo de vivienda_Casa Unifamiliar", 
    "Tipo de vivienda_Duplex", "Tipo de vivienda_Piso"
]

features = [datos_entrada[key] for key in feature_names]
features_df = pd.DataFrame([features], columns=feature_names)

scaler = StandardScaler()

features_to_normalize = ["TMEDIA", "TMIN", "TMAX", "VELMEDIA", "SOL", "PRESMAX", "PRESMIN", "Potencia contratada (kW)", "Mes", "Media de residentes"]

features_df[features_to_normalize] = scaler.fit_transform(features_df[features_to_normalize])

prediccion = modelo.predict(features_df)

prediccion = max(0, prediccion[0])

print(f"La predicción de consumo energético es: {prediccion} kWh")


La predicción de consumo energético es: 259.5109702758991 kWh


c:\Users\Alma\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

# Registrar la función de pérdida y cargar el modelo
custom_objects = {'mse': tf.keras.losses.MeanSquaredError()}
modelo_lstm = load_model("../modelos_guardados/AModelo_precios.h5", custom_objects=custom_objects)
print("Modelo cargado exitosamente.")

# **Función para calcular el precio medio en un intervalo de tiempo**
def predecir_precio_medio_intervalo(provincia, fecha_inicio, fecha_fin):
    # Generar rango de fechas
    dias_intervalo = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

    # **Filtrar datos de la provincia**
    data_provincia = data[data['Provincia'].str.upper() == provincia.upper()]

    if data_provincia.empty:
        raise ValueError(f"No se encontraron datos para la provincia {provincia}. Por favor, revisa el nombre.")

    # Obtener el último valor disponible como punto de partida
    ultimo_valor = data_provincia['Precio total con impuestos (€/MWh)'].values[-1]
    valor_inicial = np.array([[ultimo_valor]]).reshape(1, 1, 1)

    # **Predicciones para cada día en el intervalo**
    predicciones = []
    for _ in dias_intervalo:
        prediccion = modelo_lstm.predict(valor_inicial).flatten()[0]
        predicciones.append(max(prediccion, 0))  # Asegurar que no haya valores negativos
        valor_inicial = np.array([[prediccion]]).reshape(1, 1, 1)

    # **Calcular el precio medio**
    precio_medio_intervalo = np.mean(predicciones)
    return precio_medio_intervalo

# **Cargar el dataset**
file_path = "../Limpieza_datos/Modelo_Precios_Met_Fest.csv"
data = pd.read_csv(file_path, delimiter=';')
data['FECHA'] = pd.to_datetime(data['FECHA'])

# **Ejemplo de uso**
provincia = "ALAVA"  # Provincia
fecha_inicio = "2025-01-01"  # Fecha de inicio
fecha_fin = "2025-01-31"  # Fecha de fin

try:
    precio_medio = predecir_precio_medio_intervalo(provincia, fecha_inicio, fecha_fin)
    print(f"Precio medio estimado para el intervalo {fecha_inicio} a {fecha_fin} en {provincia}: {precio_medio:.2f} €/MWh")
except ValueError as e:
    print(e)


Modelo cargado exitosamente.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━

In [3]:
print("Input shape:", modelo_lstm.input_shape)
modelo_lstm.summary()


Input shape: (None, 1, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 100)         │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,053 (277.55 KB)

 Trainable params: 71,051 (277.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
print("Input shape:", modelo_lstm.input_shape)


Input shape: (None, 1, 1)


In [5]:
import numpy as np
# Último precio conocido, proveniente del histórico de la provincia
ultimo_valor = 100  
# Formato requerido: (batch_size, timesteps, features)
entrada = np.array([[[ultimo_valor]]])
print("Forma de entrada:", entrada.shape)  # Esto debería imprimir (1, 1, 1)

# Realizas la predicción:
prediccion = modelo_lstm.predict(entrada)
print("Predicción del siguiente precio:", prediccion)


Forma de entrada: (1, 1, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
Predicción del siguiente precio: [[100.04314]]
